#### Import Julia packages

In [109]:
# import Pkg
# Pkg.add("JSON3")
# Pkg.add("Interpolations")

In [110]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
using Interpolations

#### Import Custom functions

In [111]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

get_control_int (generic function with 1 method)

#### Read JSON File

In [112]:
width, height, windows, doors, heating_elem = get_room("Boundary.json")

(5, 3, Any[(0.75125, 2.32125, "Left"), (0.73125, 2.32125, "Right")], Any[(0.235, 0.655, "Top")], [[2.005, 3.435, 0.12125, 0.20125], [1.765, 3.215, 2.82125, 2.89125]])

In [113]:
price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")

(Any[0.01669, 0.01319, 0.00967, 0.01037, 0.02006, 0.02314, 0.01764, 0.01828, 0.01869, 0.01863, 0.01817, 0.01777], Any[0.19048, 0.15054, 0.11036, 0.11835, 0.22894, 0.2641, 0.20133, 0.20863, 0.21331, 0.21262, 0.20737, 0.20281], Any[282.15, 282.34999999999997, 282.25, 282.34999999999997, 282.15, 281.45, 280.65, 280.84999999999997, 280.75, 280.45, 280.25, 279.95], Any[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

#### Boundary conditions

In [114]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

f_Γ_wall (generic function with 1 method)

#### Domain & Discretization

In [115]:
domain = (0, width , 0, height)
partition = (25,25)
model = CartesianDiscreteModel(domain,partition) 
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

# fig = CairoMakie.plot(Ω)
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) 
# display(fig)

UnconstrainedFESpace()

#### Time parameters

In [116]:
t0 = 0.0    # Start time
tF = 28000/2   # End time
Δt = 10.0   # Timestep

10.0

#### Room parameters

In [117]:
timescale = 2
h_wall(x) = 0.22*timescale*0.05            # [W/m^2K]
h_window(x) = 1.2*timescale*0.05
h_door(x) = 1.0*timescale*0.05

ρ(x)=1.225
c(x)=1020.0
k(x)=15*timescale

h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 

#Toutdoor(x,t)=273.0
#Tout(t)=x->Toutdoor(x,t)

Tout = get_temp_function(T_out, tF)

# price = t->1-χ(t,2,8)

Tini(x)=297.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(297.0, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 625
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 6100148612406583128

#### Controll parameter

In [118]:
#q_pos(x) = χ(x[1], 0.21, 1.73) * χ(x[2], 0.13, 0.16)
q_pos = heating_elements(heating_elem)
Q(x,t)=χ(t,0.0,100.0)*3000.0*q_pos(x)*0.0
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [ ]:
L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))
L2skp(u)=(tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u)


Proj(a,b,z) = min(max(a,z),b)
a=0.0
b=1000.0*timescale
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]

Proj (generic function with 2 methods)

#### Test Gradient DEs

In [ ]:
price = get_price_function(price_NOK, tF, 1/timescale*0.0)
γ = 5000.0

(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=5,
armijoparas=(ρ=0.1, α_0=1, α_min=1e-5, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=4.656579518323541e6, k = 1
α = 1, new_cost = 4.656579518323541e6, L2fgrad = 2.2107685777712777e7, interm = 0.0
entered for loop, E=2.3671043180023464e6, k = 2
α = 1, new_cost = 2.3671043180023464e6, L2fgrad = 1.5762231360335307e7, interm = 0.0
entered for loop, E=1.2033124844604237e6, k = 3
α = 1, new_cost = 1.2033124844604237e6, L2fgrad = 1.1238127204010606e7, interm = 0.0
entered for loop, E=621555.3635615098, k = 4
α = 1, new_cost = 621555.3635615098, L2fgrad = 8.0766769213997265e6, interm = 0.0
entered for loop, E=335206.92163202993, k = 5
α = 1, new_cost = 335206.92163202993, L2fgrad = 5.930933196513598e6, interm = 0.0


(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (50.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (70.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (90.0, SingleFieldFEFunction())  …  (13910.0, SingleFieldFEFunction()), (13920.0, SingleFieldFEFunction()), (13930.0, SingleFieldFEFunction()), (13940.0, SingleFieldFEFunction()), (13950.0, SingleFieldFEFunction()), (13960.0, SingleFieldFEFunction()), (13970.0, SingleFieldFEFunction()), (13980.0, SingleFieldFEFunction()), (13990.0, SingleFieldFEFunction()), (14000.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction(

In [134]:
price = get_price_function(price_NOK, tF, 1/timescale*10)
γ = 1e10

(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=qs, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=3,
armijoparas=(ρ=0.1, α_0=0.001, α_min=1e-7, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=1.862709462269693e13, k = 1
α = 0.01, new_cost = 1.862709462269693e13, L2fgrad = 8.843074261750662e13, interm = 0.0
α = 0.001, new_cost = 2.979478895391136e14, L2fgrad = 8.843074261750662e13, interm = 0.0
α = 0.0001, new_cost = 2.979478895391136e14, L2fgrad = 8.843074261750662e13, interm = 0.0
α = 1.0e-5, new_cost = 2.979478895391136e14, L2fgrad = 8.843074261750662e13, interm = 0.0
α = 1.0000000000000002e-6, new_cost = 2.6133190326801753e14, L2fgrad = 8.843074261750662e13, interm = 0.0
entered for loop, E=1.6657666225131209e13, k = 2
α = 0.01, new_cost = 1.6657666225131209e13, L2fgrad = 8.361118733966088e13, interm = 0.0
α = 0.001, new_cost = 1.8626318073294164e13, L2fgrad = 8.361118733966088e13, interm = 0.0
α = 0.0001, new_cost = 1.8626318073294164e13, L2fgrad = 8.361118733966088e13, interm = 0.0
α = 1.0e-5, new_cost = 1.8626318073294164e13, L2fgrad = 8.361118733966088e13, interm = 0.0
α = 1.0000000000000002e-6, new_cost = 1.8626318073294164e

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction()), (50.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (70.0, SingleFieldFEFunction()), (80.0, SingleFieldFEFunction()), (90.0, SingleFieldFEFunction())  …  (13910.0, SingleFieldFEFunction()), (13920.0, SingleFieldFEFunction()), (13930.0, SingleFieldFEFunction()), (13940.0, SingleFieldFEFunction()), (13950.0, SingleFieldFEFunction()), (13960.0, SingleFieldFEFunction()), (13970.0, SingleFieldFEFunction()), (13980.0, SingleFieldFEFunction()), (13990.0, SingleFieldFEFunction()), (14000.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (10.0, SingleFieldFEFunction()), (20.0, SingleFieldFEFunction()), (30.0, SingleFieldFEFunction()), (40.0, SingleFieldFEFunction(

In [ ]:
price = get_price_function(price_NOK, tF, 1/timescale*10)
γ = 1e10

(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=qs, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=10,
armijoparas=(ρ=0.1, α_0=0.001, α_min=1e-7, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=1.862703172503071e13, k = 1
α = 0.001, new_cost = 1.862703172503071e13, L2fgrad = 8.843074269550086e13, interm = 0.0
α = 0.0001, new_cost = 2.979478895391136e14, L2fgrad = 8.843074269550086e13, interm = 0.0
α = 1.0e-5, new_cost = 2.979478895391136e14, L2fgrad = 8.843074269550086e13, interm = 0.0
α = 1.0000000000000002e-6, new_cost = 2.633847301631295e14, L2fgrad = 8.843074269550086e13, interm = 0.0
α = 1.0000000000000002e-7, new_cost = 1.9743522262079258e13, L2fgrad = 8.843074269550086e13, interm = 0.0
entered for loop, E=2.986684957966273e11, k = 2
α = 0.001, new_cost = 2.986684957966273e11, L2fgrad = 1.1158866731567521e13, interm = 0.0
α = 0.0001, new_cost = 1.8626318073294164e13, L2fgrad = 1.1158866731567521e13, interm = 0.0
α = 1.0e-5, new_cost = 1.8626318073294164e13, L2fgrad = 1.1158866731567521e13, interm = 0.0
α = 1.0000000000000002e-6, new_cost = 1.5725421478677363e13, L2fgrad = 1.1158866731567521e13, interm = 0.0
entered for loop, E=8

In [ ]:
# Opprett en mappe for midlertidige filer hvis den ikke eksisterer
if !isdir("tmp")
    mkdir("tmp")
end

# Initialtilstanden og resten av tidsløsningene
# uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
# uh  = solution[2:end]  # De resterende løsningene


uh = Ts
# Lagre resultater i Paraview-format (VTK)
createpvd("results") do pvd
    # Første løsning (t=0)
    # pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

    # Lagrer løsninger for hver tidssteg
    for (tn, uhn) in uh
        pvd[tn] = createvtk(Ω, "tmp/results_$tn.vtu", cellfields=["u" => uhn])
    end
end


In [ ]:
T = get_temperature_int(Ts, width, height)
q = get_control_int(qs)
result_plot()